In [7]:
from neo4j import GraphDatabase
import json

# Load data
with open("test_ant.json", "r", encoding="utf-8") as f:
    data = json.load(f)

entities = data["entities"]
relations = data["relations"]

# Label mapping for node types
label_map = {
    "character": "Human",
    "ai": "AI",
    "organization": "Organization",
    "location": "Location",
    "weapon": "Weapon",
    "vehicle": "Vehicle",
    "timelines": "Timeline"
}

# Connect to Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Create entity node
def create_entity(tx, name, label):
    cypher_label = label_map.get(label, "Entity")
    query = f"""
    MERGE (n:{cypher_label} {{name: $name}})
    SET n.type = $label_type, n.confidence = $confidence
    """
    tx.run(query, name=name, label_type=label.lower(), confidence=0.9)


# Create relationship (assumes subject & object nodes exist or are already merged)
def create_relation(tx, subject, predicate, obj):
    # Safely handle predicates with special characters by wrapping them in backticks
    predicate = f"`{predicate.upper().replace(' ', '_')}`"
    query = f"""
    MERGE (a {{name: $subject}})
    MERGE (b {{name: $object}})
    MERGE (a)-[r:{predicate}]->(b)
    """
    tx.run(query, subject=subject, object=obj)


with driver.session(database="test3") as session:
    # Load Entities
    for entity in entities:
        session.write_transaction(create_entity, entity["text"], entity["label"])


    # # Load All Relations (no validation)
    for rel in relations:
        session.write_transaction(create_relation, rel["subject"], rel["predicate"], rel["object"])

driver.close()


C:\Users\Thakk\AppData\Local\Temp\ipykernel_30304\4192165398.py:50: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_entity, entity["text"], entity["label"])
C:\Users\Thakk\AppData\Local\Temp\ipykernel_30304\4192165398.py:55: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relation, rel["subject"], rel["predicate"], rel["object"])


In [2]:
from neo4j import GraphDatabase
import json

# Load JSON file
with open("endgame.json", "r", encoding="utf-8") as f:
    data = json.load(f)

relations = data.get("relations", [])

# Neo4j setup
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Create relation function
def create_relation(tx, subject, predicate, object_):
    query = """
    MERGE (a {name: $subject})
    MERGE (b {name: $object})
    MERGE (a)-[r:%s]->(b)
    RETURN a, r, b
    """ % predicate.upper().replace(" ", "_")  # convert to uppercase and safe format
    tx.run(query, subject=subject, object=object_)

# Feed into Neo4j
with driver.session() as session:
    for rel in relations:
        session.write_transaction(
            create_relation,
            rel["subject"],
            rel["predicate"],
            rel["object"]
        )

driver.close()


C:\Users\Thakk\AppData\Local\Temp\ipykernel_36300\2750105068.py:26: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
